In [ ]:
# Conference paper_2D CNN outperforms 3D CNN in Small Dataset Classification

# 2D and 3D CNN models were trained on a personal computer (NVIDIA TITAN RTX GPU, Python 3.7.9, TensorFlow 2.1.0)

In [ ]:
from __future__ import division, print_function, absolute_import

import math
import time
import sys
import cv2
import os
import random
import itertools
from pathlib import Path

import numpy as np
from PIL import Image
from os import listdir
from os.path import join, basename, isdir
from scikitplot.metrics import plot_confusion_matrix, plot_roc
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from skimage.color import gray2rgb
from tqdm.notebook import trange, tqdm, tqdm_notebook
import scipy
from scipy import ndimage
import h5py
import zipfile
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as im
import matplotlib.image as img
import nibabel as nb
from nibabel.testing import data_path
import SimpleITK as sitk

%matplotlib inline

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
# If you use a recent tensorflow, you HAVE TO use the tensorflow.keras exclusively, or you can get weird results from version mismatches.

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, save_model, Sequential   #to creat hirachical layers
from tensorflow.keras.layers import Dense, Activation, Dropout,SpatialDropout2D, Flatten, Conv3D, Conv2D, MaxPool3D, MaxPool2D, GlobalAveragePooling3D, GlobalAveragePooling2D, Flatten, Input, BatchNormalization, GRU, Bidirectional,Reshape
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback
from tensorflow.keras.losses import categorical_crossentropy, binary_crossentropy
from tensorflow.keras.optimizers import Adadelta, Adam, SGD
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.utils import to_categorical
from matplotlib.pyplot import cm
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.


In [ ]:
# Confusion matrix
class_labels = ['Newborn', '12 Months', '24 Months', '36 Months']

def plot_confusion_matrix(cm,
                          target_names=class_labels,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        #plt.colorbar()
        #plt.imshow(data, cmap=cmap, vmin=0, vmax=1) 
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # [mwen]

    thresh = cm.max() / 2 # [mwen]
    thresh_norm = cm_norm.max() / 1.5 # [mwen]
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm_norm[i, j]) + "\n{:,}".format(cm[i, j]), # [mwen]
                     horizontalalignment="center", verticalalignment="center", fontsize=12,
                     color="white" if cm_norm[i, j] > thresh_norm else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label', fontsize=12, color='darkblue') # fontweight='bold'
    plt.xlabel('Predicted label\nAccuracy={:0.2f}; Misclassification rate={:0.2f}'.format(accuracy, misclass), fontsize=12, color='darkblue')
    plt.show()

In [ ]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
# Global definitions
DO_3D = False

if DO_3D:
    INPUT_SHAPE = (150,150,20,3) 
else:
    INPUT_SHAPE = (150,150,3)

NUM_CLASSES = -1 # will be determined from the number of folders later.

In [ ]:

DATA_DIR = "F:/Python/Dataset/NIMH/Fusion/dev"  # Directory of dataset



In [ ]:
os.listdir(DATA_DIR)

In [ ]:

# These are needed to decide based on the file path if an image belongs to train or valid
TRAIN_DIR_NAME = 'train_part'
VALID_DIR_NAME = 'valid_part'

### Create a dataframe to hold the training/validation data information.

In [ ]:

source_files = []
for current_file in Path(DATA_DIR).rglob('*.gz'):
    source_files.append(str(current_file).split('\\')[-3:])
source_files_df = pd.DataFrame(source_files)

print(source_files_df.head()) # Add this line to receive the output. It has to have three columns, or the format has to be fixed.

#source_files_df.columns = ['a', 'b', 'c', 'd', 'set', 'class', 'filename']
source_files_df.columns = ['set', 'class', 'filename']
filename_parts = source_files_df['filename'].str.split('_', expand=True) # Note that this is NOT the str.split() method, but a Pandas version! expand=True yields dataframe columns rather than a list.
source_files_df['patient_id'] = filename_parts[1]
source_files_df['visit'] = filename_parts[2]
ending = filename_parts[3].str.split('.', expand=True)
source_files_df['contrast'] = ending[0]
source_files_df.head()

CATEGORIES = list(source_files_df['class'].unique())
NUM_CLASSES = len(CATEGORIES)

# Create dictionary of target classes
label_dict = {
 0: 'Month0',
 1: 'Month12',
 2: 'Month24',
 3: 'Month36',
}

In [ ]:
FOLDS = 5 # How many folds to split into

patient_ids = source_files_df.patient_id.unique()

# For stratified folds, we need to figure out the class for each patient. We assign it to the first, if imaged multiple times.
patient_classes = []
for patID in patient_ids:
    patient_classes.append(source_files_df.loc[source_files_df['patient_id']==patID]['class'].unique()[0])
#print(list(zip(patient_ids, patient_classes)))

kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=4711)

# Save the folds yielded by the generator method into an array, in order not to need to wrap the entire training into the folds.
folds = []
for train_indices, val_indices in kf.split(patient_ids, patient_classes):
    folds.append([patient_ids[train_indices], patient_ids[val_indices]])

# Create 2D  and 3D dataset

In [ ]:
pred = []
history = []

for TRAIN_ON_FOLD in range(FOLDS):

    if DO_3D:
        RESCALE_SHAPE = (250,250,30) # From this size, we will discard 50,50,5 from each border to yield the 150x150x20 size.    
    else:
        RESCALE_SHAPE = (250,250,50) # Don't throw away so much information for 2D
    INTERPOL_ORDER = 2

    train_data = []
    train_labels = []
    valid_data = []
    valid_labels = []

    # loop over patients.
    for patient in tqdm_notebook(patient_ids[:], desc='Patient'): # First, go by patients.
        images = source_files_df.loc[source_files_df['patient_id']==patient]
        # print(patient, '\n', images)
        classes = images['class'].unique()

        # loop over classes this patient is in.
        for the_class in classes: # Then by class (age group)
            class_images = images.loc[images['class']==the_class]
            #print(the_class, '\n', class_images)
            visits = class_images['visit'].unique()

            # Make sure there aren't multiple visits.
            for visit in visits: # Then by visits...
                visit_images = class_images.loc[class_images['visit']==visit]
                #print(visit, '\n', class_images)

                # For the fusion data, we expect 3 contrasts (which will end up in the channel dimension)
                if len(visit_images)!=3:
                    print('should always have three contrasts, but have %d for patient %s, class %s' %(len(visit_images),patient,the_class))
                    continue;

                if DO_3D: 
                    image_out_size = (3,150,150,20)
                else:
                    image_out_size = (3,150,150) # We will discard the border to save space.

                multichannel_image = np.zeros(image_out_size)
                multichannel_image_flipped = np.zeros(image_out_size)
                multichannel_image_rotL = np.zeros(image_out_size)
                multichannel_image_rotR = np.zeros(image_out_size)
                multichannel_image_rand_rotL = np.zeros(image_out_size)
                multichannel_image_rand_rotR = np.zeros(image_out_size)

                # now read, preprocess, cut, average, and concatenate the images of one visit
                for idx, (index,image) in enumerate(visit_images.iterrows()):
                    #print(os.path.join(DATA_DIR,image['set'],image['class'],image['filename']))
                    nii_data = nb.load(os.path.join(DATA_DIR,image['set'],image['class'],image['filename'])) # compose the filename from the current dataframe entry
                    np_data = np.asarray(nii_data.dataobj)

                    # resize data using ndimage from scipy (https://docs.scipy.org/doc/scipy/reference/ndimage.html)
                    original_shape = np_data.shape
                    scale = [(RESCALE_SHAPE[i] + 0.0)/original_shape[i] for i in range(len(original_shape))]
                    out_data = ndimage.interpolation.zoom(np_data, scale, mode= "nearest", order = INTERPOL_ORDER)
                    out_data_mean = np.mean(out_data[50:200,50:200,int(out_data.shape[2]/2-2):int(out_data.shape[2]/2+2)], axis=2) # Select some slices in the middle and average.
                    rand_rotL = np.random.randint(10,90)
                    rand_rotR = np.random.randint(10,90)
                    if DO_3D:
                        multichannel_image[idx] = cv2.normalize(out_data[50:200,50:200,5:25], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                        if patient in folds[TRAIN_ON_FOLD][0]: # Only augment training data
                            multichannel_image_flipped[idx] = cv2.normalize(out_data[50:200,50:200,5:25], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                            multichannel_image_rotL[idx] = cv2.normalize(scipy.ndimage.rotate(out_data[50:200,50:200,5:25],-10, reshape=False), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                            multichannel_image_rotR[idx] = cv2.normalize(scipy.ndimage.rotate(out_data[50:200,50:200,5:25], 10, reshape=False), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                            multichannel_image_rand_rotL[idx] = cv2.normalize(scipy.ndimage.rotate(out_data[50:200,50:200,5:25],-rand_rotL, reshape=False), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                            multichannel_image_rand_rotR[idx] = cv2.normalize(scipy.ndimage.rotate(out_data[50:200,50:200,5:25], rand_rotR, reshape=False), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                    else:
                        multichannel_image[idx] = cv2.normalize(out_data_mean, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                        if patient in folds[TRAIN_ON_FOLD][0]: # Only augment training data
                            multichannel_image_flipped[idx] = cv2.normalize(out_data_mean[::-1], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                            multichannel_image_rotL[idx] = cv2.normalize(scipy.ndimage.rotate(out_data_mean, -10, reshape=False), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                            multichannel_image_rotR[idx] = cv2.normalize(scipy.ndimage.rotate(out_data_mean, 10, reshape=False), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                            multichannel_image_rand_rotL[idx] = cv2.normalize(scipy.ndimage.rotate(out_data_mean, -rand_rotL, reshape=False), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                            multichannel_image_rand_rotR[idx] = cv2.normalize(scipy.ndimage.rotate(out_data_mean, rand_rotR, reshape=False), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

                # Assign to either train or valid data
                if patient in folds[TRAIN_ON_FOLD][0]:
                    train_data.append([np.moveaxis(multichannel_image,0,3 if DO_3D else 2), CATEGORIES.index(the_class)])  # add this to our train_data            
                    train_data.append([np.moveaxis(multichannel_image_flipped,0,3 if DO_3D else 2), CATEGORIES.index(the_class)]) 
                    train_data.append([np.moveaxis(multichannel_image_rotL,0,3 if DO_3D else 2), CATEGORIES.index(the_class)])
                    train_data.append([np.moveaxis(multichannel_image_rotR,0,3 if DO_3D else 2), CATEGORIES.index(the_class)])
                    train_data.append([np.moveaxis(multichannel_image_rand_rotL,0,3 if DO_3D else 2), CATEGORIES.index(the_class)])
                    train_data.append([np.moveaxis(multichannel_image_rand_rotR,0,3 if DO_3D else 2), CATEGORIES.index(the_class)])
                else:
                    valid_data.append([np.moveaxis(multichannel_image,0,3 if DO_3D else 2), CATEGORIES.index(the_class)])
                    
                    
    
    # Create traain and valid
    X_train = []
    y_train = []

    for features,label in train_data:
        X_train.append(features)
        y_train.append(label)

    if DO_3D:
        X_train = np.array(X_train).reshape(-1, INPUT_SHAPE_3D[0], INPUT_SHAPE_3D[1], INPUT_SHAPE_3D[2], INPUT_SHAPE_3D[3])
    else:
        X_train = np.array(X_train).reshape(-1, INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2])
    y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
    print ("x_train shape: ", X_train.shape)
    print ("y_train shape: ", y_train.shape)

    X_valid = []
    y_valid = []

    for features,label in valid_data:
        X_valid.append(features)
        y_valid.append(label)

    if DO_3D:
        X_valid = np.array(X_valid).reshape(-1, INPUT_SHAPE_3D[0], INPUT_SHAPE_3D[1], INPUT_SHAPE_3D[2], INPUT_SHAPE_3D[3])
    else:
        X_valid = np.array(X_valid).reshape(-1, INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2])
    y_valid = to_categorical(y_valid, num_classes=NUM_CLASSES)
    print ("x_test shape: ", X_valid.shape)
    print ("y_test shape: ", y_valid.shape)

    
    np.save('xtrain-aug_%d.npy'%TRAIN_ON_FOLD, X_train)
    np.save('xvalid_%d.npy'%TRAIN_ON_FOLD, X_valid)
    np.save('ytrain-aug_%d.npy'%TRAIN_ON_FOLD, y_train)
    np.save('yvalid_%d.npy'%TRAIN_ON_FOLD, y_valid)    

####################################################################################################################    
    #CNN Model
    from keras.constraints import max_norm
    from tensorflow.keras.layers import Dense, Activation, Dropout,SpatialDropout2D, SpatialDropout3D

    num_classes=4



    def Conv(filters=16, kernel_size=(3,3), activation='relu', input_shape=None):
        if input_shape:
            return Conv2D(filters=filters, kernel_size=kernel_size, 
                          activation=activation, input_shape=input_shape) 
        else:
            return Conv2D(filters=filters, kernel_size=kernel_size,
                          activation=activation)

    def Conv_3d(filters=16, kernel_size=(3,3,3), activation='relu', input_shape=None):
        if input_shape:
            return Conv3D(filters=filters, kernel_size=kernel_size, 
                          activation=activation, input_shape=input_shape)
        else:
            return Conv3D(filters=filters, kernel_size=kernel_size,
                          activation=activation)


    # Define 2D Model
    def CNN(input_dim, num_classes): 
        model = Sequential()

        model.add(Conv(64, (3,3), input_shape=(150,150,3)))
        model.add(Conv(64, (3,3)))
        model.add(BatchNormalization())
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(Dropout(0.2))    
 

        model.add(Conv(64, (3,3)))
        model.add(Conv(64, (3,3)))
        model.add(BatchNormalization())
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(Dropout(0.2))
        

        model.add(Conv(128, (3,3)))
        model.add(Conv(128, (3,3)))
        model.add(BatchNormalization())
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(Dropout(0.2))

        model.add(GlobalAveragePooling2D())
        #model.add(Flatten())

        model.add(Dense(128, activation='relu',  kernel_constraint=max_norm(2.)))
        model.add(Dropout(rate=0.7))
        model.add(Dense(128, activation='relu',  kernel_constraint=max_norm(2.)))
        model.add(Dropout(0.2))

        model.add(Dense(num_classes, activation='softmax'))

        return model

###################################################################################################       
    # Define 3D Model
    def CNN_3d(input_shape = (150,150,20,3), num_classes=4):
        model = Sequential()

        model.add(Conv_3d(32, input_shape=(150,150,20,3)))
        model.add(Conv_3d(32))
        model.add(BatchNormalization())
        model.add(MaxPool3D(pool_size=(2,2,2)))
        #model.add(MaxPool3D(pool_size=(2)))
        model.add(MaxPool3D())
        model.add(Dropout(0.2))    

        model.add(Conv_3d(64))
        model.add(Conv_3d(64))
        model.add(BatchNormalization())
        model.add(MaxPool3D(pool_size=(2,2,2)))
        model.add(Dropout(0.2))

        model.add(Conv_3d(96))
        model.add(Conv_3d(96))
        model.add(BatchNormalization())
        model.add(MaxPool3D(pool_size=(2,2,2)))
        model.add(Dropout(0.2))

        model.add(Conv_3d(128))
        #model.add(BatchNormalization())
        #model.add(MaxPool3D())
        #model.add(Dropout(0.2))


        model.add(GlobalAveragePooling3D())
        #model.add(Flatten())

        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.7))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))


        model.add(Dense(num_classes, activation='softmax'))

        return model

##################################################################################################


################################################################################################
    # 2D variants
    model = CNN(INPUT_SHAPE, NUM_CLASSES)

    # 3D variants
    #model = CNN_3d(INPUT_SHAPE_3D, NUM_CLASSES)

    trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
    non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])
    print('Total params: {:,}'.format(trainable_count + non_trainable_count))
    print('Trainable params: {:,}'.format(trainable_count))
    print('Non-trainable params: {:,}'.format(non_trainable_count))    
    
####################################################################################################
    optimizer = tf.keras.optimizers.Adam(lr=0.001)

    # Make loss more expensive for the tougher classes.

    earlystop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True)
    model.compile(loss=categorical_crossentropy,  optimizer= optimizer , metrics=['acc'])

    history.append(model.fit(X_train, y_train, batch_size=8, epochs=300, validation_data=(X_valid,y_valid), class_weight=class_weight, callbacks=[earlystop_cb]))

##################################################################################################################


    #Confusion Matrix and Classification Report

    pred.append(model.predict(X_valid, batch_size=8)) 


#### Apart from now postprocessing the predictions for the desired fold, nothing has to change...

In [ ]:

# Confusion Matrix and Classification Report for individual folds
y_true_all = []
y_pred_all = []
for WHICH_FOLD in range(FOLDS):

    y_pred = np.argmax(pred[WHICH_FOLD], axis=1)
    y_pred_all.append(y_pred)
    # Load corresponding y_valid data
    y_true = np.load(str('yvalid_%d.npy'%WHICH_FOLD))

    y_true = np.argmax(y_true, axis=1)
    y_true_all.append(y_true)

    class_labels = ['Newborn', '12 Months','24 Months', '36 Months']
    print(classification_report(y_true, y_pred, target_names=class_labels))
    print(confusion_matrix(y_true, y_pred))

In [ ]:
# Calculate summary statistics from all validation runs    
flat_y_pred_all = [item for sublist in y_pred_all for item in sublist]
flat_y_true_all = [item for sublist in y_true_all for item in sublist]
cm = confusion_matrix(flat_y_true_all, flat_y_pred_all)
class_labels = ['Newborn', '12 Months','24 Months', '36 Months']
print("Summed Metrics:")
print(classification_report(flat_y_true_all, flat_y_pred_all, target_names=class_labels))
print(cm)

In [ ]:
plot_confusion_matrix(cm, normalize=True)

In [ ]:
from pycm import *

cm = ConfusionMatrix(flat_y_true_all, flat_y_pred_all)
cm.table
print(cm)

In [ ]:
#Plot Accuracy and los for 5fold 
losses = []
val_losses   = []
accs = []
val_accs   = []
for hist in history:
    losses.append(hist.history['loss'])
    val_losses.append(hist.history['val_loss'])
    accs.append(hist.history['acc'])
    val_accs.append(hist.history['val_acc'])

shortest = min([len(l) for l in losses])
print ('Shortest training:', shortest)

nlosses = [l[:shortest] for l in losses]
nval_losses = [l[:shortest] for l in val_losses]
naccs = [l[:shortest] for l in accs]
nval_accs = [l[:shortest] for l in val_accs]

mean_l = np.mean(nlosses, axis=0)
mean_vl = np.mean(nval_losses, axis=0)
mean_a = np.mean(naccs, axis=0)
mean_va = np.mean(nval_accs, axis=0)
min_l = np.min(nlosses, axis=0)
min_vl = np.min(nval_losses, axis=0)
min_a = np.min(naccs, axis=0)
min_va = np.min(nval_accs, axis=0)
max_l = np.max(nlosses, axis=0)
max_vl = np.max(nval_losses, axis=0)
max_a = np.max(naccs, axis=0)
max_va = np.max(nval_accs, axis=0)

# Visualize the result
plt.figure(figsize=(12, 6))
plt.plot(mean_l, '-r', label="Train loss")
plt.fill_between(range(shortest), min_l, max_l,
                 color='lightcoral', alpha=0.4)
plt.plot(mean_vl, '-b', label="Val loss")
plt.fill_between(range(shortest), min_vl, max_vl,
                 color='cornflowerblue', alpha=0.4)
plt.legend()
plt.title("Train/Valid Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')


plt.figure(figsize=(12, 6))
plt.plot(mean_a, '-r', label="Train acc")
plt.fill_between(range(shortest), min_a, max_a,
                 color='lightcoral', alpha=0.4)
plt.plot(mean_va, '-b', label="Val acc")
plt.fill_between(range(shortest), min_va, max_va,
                 color='cornflowerblue', alpha=0.4)
plt.legend()
plt.title("Train/Valid Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [ ]:
# If you want to plot one specific fold
# plot Loss
WHICH_FOLD=1
plt.figure(figsize=(12, 6))
plt.plot(history[WHICH_FOLD].history['loss'], label="Train loss")
plt.plot(history[WHICH_FOLD].history['val_loss'], label="Val loss")
plt.legend()
plt.title("Train/Valid Loss (fold %d)"%WHICH_FOLD)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
# plot accuracy for one specific fold
WHICH_FOLD=1
plt.figure(figsize=(12, 6))
plt.plot(history[WHICH_FOLD].history['acc'], label="Train acc")
plt.plot(history[WHICH_FOLD].history['val_acc'], label="Val acc")
plt.legend()
plt.title("Train/Valid Acc (fold %d)"%WHICH_FOLD)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# Statistical results
from pycm import *

cm = ConfusionMatrix(flat_y_true_all, flat_y_pred_all)
cm.table
print(cm)